In [ ]:
from madmex.util import parser_extra_args, join_dicts
from madmex.models import ChangeInformation
from madmex.wrappers import gwf_query
from importlib import import_module
from datacube.api import GridWorkflow
import numpy
#from madmex.lcc.transform.mad import Transform as MAD
from scipy import stats

In [ ]:
def transform_mad(X, Y, lmbda=0.0, weights=None, bands=1, rows=5002, cols=5002):
        #weights = weights
        if weights is None:
            weights = numpy.ones((rows, cols))
        W = weights * numpy.ones(X.shape)
        X_mean = numpy.average(X, weights=W, axis=(1,2))
        Y_mean = numpy.average(Y, weights=W, axis=(1,2))
        X_centered = (X - X_mean[:,numpy.newaxis,numpy.newaxis])
        Y_centered = (Y - Y_mean[:,numpy.newaxis,numpy.newaxis])

        X_weigthed = X_centered * weights
        Y_weigthed = Y_centered * weights
        X_pixel_band = X_centered.reshape(bands, rows * cols)
        Y_pixel_band = Y_centered.reshape(bands, rows * cols)

        X_pixel_band_weigthed = X_weigthed.reshape(bands, rows * cols)
        Y_pixel_band_weigthed = Y_weigthed.reshape(bands, rows * cols)

        sigma_11 = numpy.matmul(X_pixel_band_weigthed, X_pixel_band.T) / (numpy.sum(weights) - 1)
        sigma_11 = (1-lmbda) * sigma_11 + lmbda * numpy.eye(bands)
        sigma_22 = numpy.matmul(Y_pixel_band_weigthed, Y_pixel_band.T) / (numpy.sum(weights) - 1)
        sigma_22 = (1-lmbda) * sigma_22 + lmbda * numpy.eye(bands)
        sigma_12 = numpy.matmul(X_pixel_band_weigthed, Y_pixel_band.T) / (numpy.sum(weights) - 1)
        lower_11 = numpy.linalg.cholesky(sigma_11)
        lower_22 = numpy.linalg.cholesky(sigma_22)
        lower_11_inverse = numpy.round(numpy.linalg.inv(lower_11), decimals=10)
        lower_22_inverse = numpy.round(numpy.linalg.inv(lower_22), decimals=10)
        sigma_11_inverse = numpy.linalg.inv(sigma_11)
        sigma_22_inverse = numpy.linalg.inv(sigma_22)

        eig_problem_1 = numpy.matmul(lower_11_inverse,
                                     numpy.matmul(sigma_12,
                                                  numpy.matmul(sigma_22_inverse,
                                                               numpy.matmul(sigma_12.T, lower_11_inverse.T))))
        eig_problem_1 = (eig_problem_1 + eig_problem_1.T) * 0.5
        eig_problem_2 = numpy.matmul(lower_22_inverse,
                                     numpy.matmul(sigma_12.T,
                                                  numpy.matmul(sigma_11_inverse,
                                                               numpy.matmul(sigma_12, lower_22_inverse.T))))
        eig_problem_2 = (eig_problem_2 + eig_problem_2.T) * 0.5
        eig_values_1, eig_vectors_1 = numpy.linalg.eig(eig_problem_1)
        eig_values_2, eig_vectors_2 = numpy.linalg.eig(eig_problem_2)

        eig_vectors_transformed_1 = numpy.matmul(lower_11_inverse.T, eig_vectors_1)
        eig_vectors_transformed_2 = numpy.matmul(lower_22_inverse.T, eig_vectors_2)

        sort_index_1 = numpy.flip(eig_values_1.argsort(), 0)
        sort_index_2 = numpy.flip(eig_values_2.argsort(), 0)

        vector_u = eig_vectors_transformed_1[:, sort_index_1]
        vector_v = eig_vectors_transformed_2[:, sort_index_2]

        mu = numpy.sqrt(eig_values_2[sort_index_2])
        norm_a_squared = numpy.diag(numpy.matmul(vector_u.T, vector_u))
        norm_b_squared = numpy.diag(numpy.matmul(vector_v.T, vector_v))

        variance_u = numpy.diag(1/numpy.sqrt(numpy.diag(sigma_11)))
        s = numpy.sum(numpy.matmul(variance_u, numpy.matmul(sigma_11, vector_u)),axis=0)
        vector_u = numpy.matmul(vector_u, numpy.diag(s / numpy.abs(s)))

        signs_vector = numpy.diag(numpy.dot(numpy.dot(vector_u.T, sigma_12), vector_v))
        signs = numpy.diag(signs_vector / numpy.abs(signs_vector))
        vector_v = numpy.matmul(vector_v, signs)

        U = numpy.matmul(vector_u.T, X_pixel_band)
        V = numpy.matmul(vector_v.T, Y_pixel_band)
        M = U - V
        sigma_squared = (2 - lmbda * (norm_a_squared + norm_b_squared)) / (1 - lmbda) - 2 * mu
        rho = mu * (1 - lmbda) / numpy.sqrt((1 - lmbda * norm_a_squared) * (1 - lmbda * norm_b_squared))
        return M.reshape(bands, rows, cols), sigma_squared, rho

In [ ]:
algorithm = 'imadmaf'
band_list = None
name = 'test4_JAL_2017_2018'
product_pre = 's1_2_20mres10m_JAL_recipe2017'
product_post = 's1_2_20m_resampled_10m_001_Jalisco_from_s3_to_s3_recipe_2018'
lc_pre = 'land_cover_rf_s1_2_20mres10m_JAL_recipe2017'
lc_post = 'land_cover_rf_s1_2_20m_resampled_10m_001_jalisco_2018_mxnb_31_02'
year_pre = 2017
year_post = 2018
filter_labels = True
mmu = 5000.0
extra_args = parser_extra_args('')
scheduler_file = None
#options = {'lat': [20.833, 21.125], 'long': [-103.1395, -102.796], 'region': None}
options = {'lat': [20.04, 20.401], 'long': [-103.1395, -102.796], 'region': None}
#options = {'lat': [20.04, 20.401], 'long': [-103.48275, -103.1395, 'region': None}

In [ ]:
change_meta, _ = ChangeInformation.objects.get_or_create(year_pre=year_pre,
                                                          year_post=year_post,
                                                          algorithm=algorithm,
                                                          name=name)

In [ ]:
# Build gwf_kwargs, send a query for both products, combine the dict and generate iterable
gwf_kwargs = { k: options[k] for k in ['lat', 'long', 'region']}
pre_dict = gwf_query(product_pre, view=False, **gwf_kwargs)
post_dict = gwf_query(product_post, view=False, **gwf_kwargs)
iterable = join_dicts(pre_dict, post_dict, join='inner').items()

In [ ]:
# detect and classify starts
#module = import_module('madmex.lcc.bitemporal.%s' % algorithm)
#BiChange = module.BiChange

In [ ]:
# Load geoarrays
geoarray_pre = GridWorkflow.load(list(iterable)[0][1][0], measurements=band_list)
#BiChange_pre = BiChange.from_geoarray(geoarray_pre, **extra_args)
geoarray_post = GridWorkflow.load(list(iterable)[0][1][1], measurements=band_list)
#BiChange_post = BiChange.from_geoarray(geoarray_post)

In [ ]:
# Run change detection
#BiChange_pre.run(BiChange_post)

In [ ]:
arr0 = geoarray_pre.squeeze().to_array().values
arr1 = geoarray_post.squeeze().to_array().values
#affine = Affine(*list(geoarray_pre.affine)[0:6])
#crs = geoarray_pre.crs._crs.ExportToProj4()

In [ ]:
if arr0.ndim == 2:
    arr0 = arr0[numpy.newaxis,:]
bands, rows, cols = arr0.shape
arr0.shape

In [ ]:
max_iterations=25
min_delta=0.01
lmbda=0.0

In [ ]:
# IRMAD
i = 0
delta = 1.0
old_rho = numpy.ones((arr0.shape[0]))
weights = numpy.ones((arr0.shape[1],arr0.shape[2]))
while i < max_iterations and delta > min_delta:
    print('Iteration #%s' % i)
    print('delta: %s' % delta)
    M, sigma_squared, rho = transform_mad(arr0, arr1, lmbda=lmbda, weights=weights, bands=bands, rows=rows, cols=cols)
    chi_square = numpy.tensordot(1 / sigma_squared, numpy.multiply(M, M), axes=1)
    weights = 1 - stats.chi2.cdf(chi_square, bands)
    delta = max(abs(rho - old_rho))
    old_rho = rho
    i = i + 1
M